In [3]:
from get_raw_modules import get_raw_data as grd
import re

url = 'https://www.digikala.com/search/category-notebook-netbook-ultrabook/?has_selling_stock=1&pageno={}&sortby=4'
data = grd(url)


In [45]:
len(data)

425

In [141]:
keys = [
    'وزن',
    'اندازه صفحه نمایش',
    'دقت صفحه نمایش',
    'سازنده پردازنده گرافیکی',
    'حافظه اختصاصی پردازنده گرافیکی',
    'نوع حافظه داخلی',
    'نوع حافظه RAM',
    'ظرفیت حافظه RAM',
    'سری پردازنده',
    'حافظه Cache',
    'سیستم عامل',
    'توضیحات باتری',
    'صفحه نمایش لمسی',
    'پورت Thunderbolt',
    'پورت USB 4.0',
    'پورت USB Type-C',
    'کیبورد با نور پس زمینه',
    'حسگر اثر انگشت',
    'درایو نوری'
]


In [172]:
def miner(raw_inp):
    outdict = {}
    outdict['Price'] = int(raw_inp['price'])
    for key in keys:
        if key == 'وزن':
            outdict['Mass'] = float(re.findall(r'(.+?) کیلوگرم', raw_inp[key])[0])

        elif key == 'اندازه صفحه نمایش':
            outdict['PanalSize'] = float(re.findall(r'(.+?) اینچ', raw_inp[key])[0])

        elif key == 'دقت صفحه نمایش':
            find = re.findall(
                r'(\d.+\d)\s', raw_inp[key])[0].replace(' ', '').split('x')
            outdict['PanalResolution'] = int(find[0]) * int(find[1])

        elif key == 'سازنده پردازنده گرافیکی':
            outdict['GPUBrand'] = raw_inp[key].replace(" ","")

        elif key == 'حافظه اختصاصی پردازنده گرافیکی':
            null = 'بدون حافظه\u200cی گرافیکی مجزا '
            find = raw_inp[key].replace('GB ', '000')
            find = find.replace('MB ', '')
            find = find.replace(null, '0')
            outdict['GPUCapacity'] = int(find)

        elif key == 'نوع حافظه داخلی':
            if key in raw_inp.keys():
                stg_type = ['SSD ', 'eMMC ', 'حافظه\u200cهای هیبریدی ', 'هارد دیسک ']
                find = raw_inp[key].replace(stg_type[2], '1')
                find = find.replace(stg_type[3], '2')
                find = find.replace(stg_type[0], '3')
                find = find.replace(stg_type[1], '4')
                outdict['DiskType'] = int(find)
            else:
                outdict['DiskType'] = 1

        elif key == 'نوع حافظه RAM':
            if '3' in raw_inp[key]:
                find = 3
            else:
                find = 4
            outdict['RAMType'] = find
            
        elif key == 'ظرفیت حافظه RAM':
            find = raw_inp[key].replace(' گیگابایت ', '')
            outdict['RAMCapacity'] = int(find)
            
        elif key == 'سری پردازنده':
            outdict['CPUSerie'] = raw_inp[key][:-1]

        elif key == 'حافظه Cache':
            if key in raw_inp.keys():
                find = raw_inp[key].replace(' مگابایت ', '')
                outdict['CacheCapacity'] = int(find)
            elif raw_inp['price'] < 10000000:
                outdict['CacheCapacity'] = 1
            else:
                outdict['CacheCapacity'] = 5

        elif key == 'توضیحات باتری':
            if key in raw_inp.keys():
                find = re.sub(r'[^\d\.\s]', "", raw_inp[key])
                find = re.findall(r'\s(\d[\d\.]+)\s', find)
                if find != []:
                    if float(find[0]) > 100:
                        find[0] = 30.0
                    outdict['Battery'] = float(find[0])
                else:
                    outdict['Battery'] = 35.0
            else:
                outdict['Battery'] = 35.5
                
        elif key == 'سیستم عامل':
            if key in raw_inp.keys():
                freeos = ['ندارد ', 'بدون ویندوز ', 'بدون سیستم\u200cعامل ',
                          'Ubuntu Linux ', 'FreeDOS ', 'Boot-Up DOS ']
                if raw_inp[key] in freeos:
                    find = 0
                else:
                    find = 1
                outdict['OS'] = find
            else:
                outdict['OS'] = 0

        elif key == 'صفحه نمایش لمسی':
            if key in raw_inp.keys():
                resp = ['بله ', 'خیر ']
                find = raw_inp[key].replace(resp[0], '1').replace(resp[1], '0')
                outdict['Touch'] = int(find)
            else:
                outdict['Touch'] = 0

        elif key == 'پورت Thunderbolt':
            if key in raw_inp.keys():
                resp = ['ندارد ', 'دارد ']
                find = raw_inp[key].replace(resp[0], '0').replace(resp[1], '1')
                outdict['Thunderbolt'] = int(find)
            else:
                outdict['Thunderbolt'] = 0

        elif key == 'پورت USB 4.0':
            if key in raw_inp.keys():
                resp = ['ندارد ', 'دارد ']
                find = raw_inp[key].replace(resp[0], '0').replace(resp[1], '1')
                outdict['USB4'] = int(find)
            else:
                outdict['USB4'] = 0
        
        elif key == 'پورت USB Type-C':
            if key in raw_inp.keys():
                resp = ['ندارد ', 'دارد ']
                find = raw_inp[key].replace(resp[0], '0').replace(resp[1], '1')
                outdict['TypeC'] = int(find)
            else:
                outdict['TypeC'] = 0

        elif key == 'کیبورد با نور پس زمینه':
            if key in raw_inp.keys():
                resp = ['ندارد ', 'دارد ']
                find = raw_inp[key].replace(resp[0], '0').replace(resp[1], '1')
                outdict['KeyboardLight'] = int(find)
            else:
                outdict['KeyboardLight'] = 0

        elif key == 'حسگر اثر انگشت':
            if key in raw_inp.keys():
                resp = ['ندارد ', 'دارد ']
                find = raw_inp[key].replace(resp[0], '0').replace(resp[1], '1')
                outdict['Fingerprint'] = int(find)
            else:
                outdict['Fingerprint'] = 0

        elif key == 'درایو نوری':
            if key in raw_inp.keys():
                resp = ['DVD-RW ', 'بدون درایو نوری ']
                find = raw_inp[key].replace(resp[0], '1').replace(resp[1], '0')
                outdict['DVD'] = int(find)
            else:
                outdict['DVD'] = 0
    return outdict


In [157]:
Prior_keys = ['Price', 'Mass', 'PanalSize', 'PanalResolution', 'GPUBrand', 'GPUCapacity', 'DiskType', 'RAMType', 'RAMCapacity',
              'CPUSerie', 'CacheCapacity', 'OS', 'Battery', 'Touch', 'Thunderbolt', 'USB4', 'TypeC', 'KeyboardLight', 'Fingerprint', 'DVD']


In [173]:
miner(data[300])

{'Price': 26650000,
 'Mass': 1.8,
 'PanalSize': 15.6,
 'PanalResolution': 2073600,
 'GPUBrand': 'NVIDIA',
 'GPUCapacity': 2000,
 'DiskType': 1,
 'RAMType': 4,
 'RAMCapacity': 16,
 'CPUSerie': 'Core i7',
 'CacheCapacity': 12,
 'OS': 0,
 'Battery': 42.0,
 'Touch': 0,
 'Thunderbolt': 0,
 'USB4': 0,
 'TypeC': 1,
 'KeyboardLight': 1,
 'Fingerprint': 1,
 'DVD': 0}

In [ ]:
for laptop in data:
    for key in Prior_keys:

In [ ]:
import mysql.connector

with open("setting.txt","")

print('connecting to the server...')
laptopdb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="36268232",
    database="laptop_data"
)
print('connected to the Sql!')


In [164]:
import mysql.connector
host = "localhost"
user = "root"
password = "36268232"
#with open("setting.txt", "")


def check_database(host_, user_, pass_):
    database = mysql.connector.connect(
        host = host_ ,
        user = user_ ,
        password = pass_
    )
    mycursor = database.cursor()
    mycursor.execute("SHOW DATABASES")
    if ("laptop_data",) in mycursor:
        return True
    else:
        return False


check_database(host, user, password)


connected to the Sql!

True

In [ ]:
def check_table(host_, user_, pass_):
    database = mysql.connector.connect(
        host=host_,
        user=user_,
        password=pass_
        database="laptop_data"
    )
    mycursor = database.cursor()
    mycursor.execute("SHOW TABLES")
    if ("selected_data",) in mycursor:
        return True
    else:
        return False


check_database(host, user, password)


In [ ]:
def create_database(host_, user_, pass_):
    database = mysql.connector.connect(
        host=host_,
        user=user_,
        password=pass_
    )
    mycursor = database.cursor()
    mycursor.execute("CREATE DATABASES laptop_data")


In [ ]:
def create_table(host_, user_, pass_):
    database = mysql.connector.connect(
        host=host_,
        user=user_,
        password=pass_
        database="laptop_data"
    )
    mycursor = database.cursor()
    execute = 'create table selected_data (Price int, Mass float, PanalSize float, PanalResolution int, GPUBrand varchar(255), GPUCapacity int, DiskType int, RAMType int, RAMCapacity int, CPUSerie varchar(255), CacheCapacity int, OS int, Battery float, Touch int, Thunderbolt int, USB4 int, TypeC int, KeyboardLight int, Fingerprint int, DVD int)'
    mycursor.execute(execute)


In [177]:
import os
print('welcome to laptop price predictor!')
setting_check = os.path.isfile('./setting')
if setting_check == True:
    with open('setting', 'r') as setting:
        settext = setting.read().split('\n')
    if settext[0] == 'y':
        choice = settext[0]
        host = settext[1]
        user = settext[2]
        password = settext[3]
    else:
        choice = 'n'
else:
    choice = input(
        'we use normal file saving method by default. do you want to change it to SQL? [y]/[n]')
    while True:
        if choice in ['y', 'n']:
            break
        else:
            print('Please respond correctly. try again:')
            choice = input()
    host = user = password = ""
    if choice == 'y':
        print('Please enter your server information:')
        host = input('Enter your host name: (for example localhost)')
        user = input('Enter your username: (for example root)')
        password = input('Enter your password:')
    dosave = input('Do you want to save changes and your information? [y]/[n]')
    while True:
        if dosave in ['y', 'n']:
            break
        else:
            print('Please respond correctly. try again:')
            dosave = input()
    if dosave == 'y':
        with open('setting', 'w') as setting:
            setting.write(choice+'\n'+host+'\n'+user+'\n'+password)


welcome to laptop price predictor!


In [178]:
if choice == 'y':

('localhost', 'root', '36268232')